# Contents
* [Introduction](#Introduction)
* [Installation](#Installation)
    * [Environment](#Setting-up-your-virtual-environment)
    * [Bazel](#Install-Bazel)
    * [Build](#Build-tensorflow-with-Bazel)
    * [Run DeepHOL](#Run-deephol)
* [Configuration](#Configuration)
* [Prover](#Prover)
    * [Prover Class]()
    * [Custom Constructor]()
    * [Single-Task Prover]()
* [Model Architecture](#Model-Architecture)
    * [Logistic Regression](#Logistic-regression-on-top-of-learned-token-embeddings.)
    * [2-layer-CNN](#2-layer-1D-convolutional-neural-network)
* [Selection Strategies](#Selection-Strategies)
    * [Predictions Class](#Attributes)
    * [Goal Embedding](#Goal-Embedding)
    * [Theorem Embedding](#Theorem-Embedding)
    * [Proof State Search](#Proof-State)
    * [Proof State Embedding](#Proof-State-Embedding)
    * [Proof State Encoding](#Proof-State-Encoding)
    * [Tactic Selection](#Tactic-Selection)
    * [Premise Selection](#Premise-Selection)
* [Related Work](#Related-Work)  

[Run](#Run-DeepHOL)

---
---
---

# Introduction  
  
  
## How to Use This Notebook  
First, follow the instructions below to install deephol in the same directory as this notebook.  

By reading through the rest of the documentation, you will be able to implement your own theorem prover via this notebook.  

Within this documentation are code cells where you can implement your own functions. Once you write the function, run the cell to overwrite it. You can also choose to leave them blank and deephol will fall back on default behavior.  

Use the [cell](#Run-DeepHol) at the bottom to run DeepHOL.

# Installation
## Basic Dependencies
* Anaconda Python
* gcc
* g++
* Docker


# Building from source
Follow this guide if you want to build on your local machine without using the container.  


## Setting up your virtual environment
Create and activate a conda environment.

In [ ]:
%%bash
conda create --name deephol
conda activate deephol
pip install h5py six numpy scipy wheel mock pyfarmhash grpcio
pip install keras_applications==1.0.6 keras_preprocessing==1.0.5 --no-deps

## Clone the deepmath repository  
Clone the modified deepmath repo and move this notebook into the repository.

In [ ]:
%%bash
git clone https://github.com/aahadley/deepmath-jupyter
cd deepmath-jupyter && mv ../HOLJup.ipynb .

Now refresh this notebook.  
  
  

## Install Bazel
Bazel is a build manager for tensorflow. We need to install this in order to build deepmath.

In [ ]:
%%bash
wget https://github.com/bazelbuild/bazel/releases/download/0.21.0/bazel-0.21.0-installer-linux-x86_64.sh
chmod 777 bazel-0.21.0-installer-linux-x86_64.sh
bash bazel-0.21.0-installer-linux-x86_64.sh --prefix=$HOME/bazel --user
PATH=$HOME/bazel/bin:$PATH

### Configure tensorflow
Navigate to the tensorflow dirctory and configure the following options. You can make changes here depending on your system.

In [ ]:
%%bash
cd deepmath/tensorflow
TF_IGNORE_MAX_BAZEL_VERSION=1   TF_NEED_OPENCL_SYCL=0   TF_NEED_COMPUTECPP=1   TF_NEED_ROCM=0   TF_NEED_CUDA=0   TF_ENABLE_XLA=0   TF_DOWNLOAD_CLANG=0   TF_NEED_MPI=0   TF_SET_ANDROID_WORKSPACE=0 CC_OPT_FLAGS="-march=native -Wno-sign-compare"  ./configure

Tensorflow will ask you to specify a python path. It will also give you a default option. Select the default and take note of the path it gives you.  
Now set the environment variable to that value.

In [ ]:
%%bash
export $PYTHON_BIN_PATH=$(which python)

## Build tensorflow with Bazel

This step will take awhile. go fix yourself some snacks while it works.

In [ ]:
%%bash
cd ..
bazel build -c opt //deepmath/deephol:main --define grpc_no_ares=true --python_path=$PYTHON_BIN_PATH

## Run deephol
### Download and Setup HOL-Light container
Create a docker network with

In [ ]:
%%bash
docker network create holist_net

Now run the HOL-Light image

In [ ]:
%%bash
docker run -d --network=holist_net --name=holist gcr.io/deepmath/hol-light

Now you can get the ip of the HOL-Light image with

In [ ]:
%%bash
docker network inspect holist_net

Or store it with

In [6]:
%%bash
export HNET_IP="$(docker inspect --format='{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' holist)"

sudo: no tty present and no askpass program specified


Now you can run deephol with

In [ ]:
%%bash

python bazel-bin/deepmath/deephol/main --prover_options=data/configuration/prover_options.textpb --output=proof_logs.textpbs --proof_assistant_server_address=$HNET_IP:2000

## Verify using the proof checker
You can expect some errors in the generated proofs, so you'll need to check them with HOL-Light.  

In [ ]:
%%bash 

python bazel-bin/deepmath/deephol/utilities/proof_checker --theorem_database=/data/theorem_database_v1.1.textpb \ 
--proof_logs=data/proof_logs.textpbs \ 
--out_file=data/synthetic_proofs.ml

cp data/proof_logs.textpbs ../hol-light
cd ..
docker build -f hol-light/Dockerfile_check_proofs --ulimit stack=1000000000 --tag check_proofs hol-light/
docker run check_proofs

# Configuration
Configuration options can be found in `deepmath/data/configuration/prover_options.textpb`. Available configuration options are determined by `deepmath/deephol/deephol.proto`. Whenever changes are made to `deephol.proto`, it must be compiled with: 

In [ ]:
%%bash 

python -m grpc_tools.protoc -I. --python_out=bazel-genfiles/deepmath/deephol/ deepmath/deephol/deephol.proto

---
---
---
# Prover
The task of proving a theorem is generally thought of as a tree search, where the root of
the tree is a theorem to be proved, and each leaf node corresponds to an applied tactic.
The Prover object is responsible for implementing the proof search. The default option in
DeepHOL is a breadth-first search, and DeepHOL also implements a no-backtracking search for
comparison. Other works have implemented different search algorithms for theorem proving
such as Monte-Carlo search.


Implementing a new prover in DeepHOL requires implementing a new subclass of the Prover class. All that is needed for this is to implement a custom constructor if needed, and the `prove_one` method, which takes a proof search tree, and a ProverTask as parameters, and return None when a proof is found, or an error message if the search fails. Example implementations can be found in [prover.py](https://github.com/tensorflow/deepmath/blob/master/deepmath/deephol/prover.py).

## Custom Constructor
If your prover needs a custom `__init__()`, you can implement it here.  

*note that the function in this file is called `_init_` rather than `__init__`. This is an intentional implementation detail and not a typo*

In [ ]:
 %%writefile deepmath/deephol/jup_prover/custom/constructor.py

###### your code below #####
    
def _init_(self, prover_options: deephol_pb2.ProverOptions, hol_wrapper,
               action_gen: action_generator.ActionGenerator,
               theorem_db: proof_assistant_pb2.TheoremDatabase):
    super(NoBacktrackProver, self).__init__(
        prover_options, hol_wrapper, theorem_db, single_goal=True)
    self.action_gen = action_gen

## Prove One
This is the only method that needs to be implemented. It should search the proof tree to find a single proof, and return `None` upon success, or an error message upon failure.

In [ ]:
 %%writefile deepmath/deephol/jup_prover/custom/prove_one.py

def prove_one(self, tree: proof_search_tree.ProofSearchTree,
                task: proof_assistant_pb2.ProverTask) -> Optional[Text]:
    """Searches for a proof via BFS.
    Args:
      tree: Search tree with a single goal node to be proved.
      task: ProverTask to be performed.
    Returns:
      None on success and error message on failure.
    """
    root = tree.nodes[0]
    nodes_explored = 0
    # Note that adding new node to the tree might re-enable previous nodes
    # for tactic applications, if they were marked to be ignored by
    # failing sibling nodes.
    tree.cur_index = 0
    while not self.timed_out() and not root.closed and not root.failed and (
            nodes_explored < self.options.max_explored_nodes):
        if tree.cur_index >= len(tree.nodes):
            return 'BFS: All nodes are failed or ignored.'
        node = tree.nodes[tree.cur_index]
        tree.cur_index += 1
        if node.ignore or node.failed or node.closed or node.processed:
            continue
        nodes_explored += 1
      # Note that the following function might change tree.cur_index
      # (if a node that was ignored suddenly becomes subgoal of a new
      # tactic application).
      prover_util.try_tactics(node, self.options.max_top_suggestions,
                              self.options.min_successful_branches,
                              self.options.max_successful_branches,
                              task.premise_set, self.action_gen,
                              self.prover_options.tactic_timeout_ms)
    root_status = ' '.join([
        p[0] for p in [('closed', root.closed), ('failed', root.failed)] if p[1]
    ])
    tf.logging.info('Timeout: %s root status: %s explored: %d',
                    str(self.timed_out()), root_status, nodes_explored)
    if self.timed_out():
        return 'BFS: Timeout.'
    elif root.failed:
        return 'BFS: Root Failed.'
    elif nodes_explored >= self.options.max_explored_nodes and not root.closed:
        return 'BFS: Node limit reached.'

# Model Architecture

**These models are developed for the task of premise selection:**
**classifying proof step statements as being useful or not in the context of a given proof**

• Unconditioned classification of proof steps: determining how likely a given proof is to be
useful for the proof it occurred in, based solely on the content of statement (i.e. by only
providing the model with the step statement itself, absent any context).

• Conditioned classification of proof steps: determining how likely a given proof is to be
useful for the proof it occurred in, with “conditioning” on the conjecture statement that the
proof was aiming to attain, i.e. by providing the model with both the step statement and the
conjecture statement).

## Logistic regression on top of learned token embeddings. 
This minimal model aims to determine to which extent simple differences between token distribution between useful and non-useful statements can be used to distinguish them. It provides an absolute floor on the performance achievable on this task.

In [ ]:
%%writefile deepmath/deephol/jup_models/custom/conditioned_model.py

"""Models for classifying proof steps with conditioning on the conjecture."""

###### your code below #####

def conditioned_model(voc_size, max_len, dropout=0.5):
      """Two siamese branches, each embedding a statement.
      Binary classifier on top.
      Args:
        voc_size: size of the vocabulary for the input statements.
        max_len: maximum length for the input statements.
        dropout: Fraction of units to drop.
      Returns:
        A Keras model instance.
      """
    pivot_input = layers.Input(shape=(max_len,), dtype='int32')
    statement_input = layers.Input(shape=(max_len,), dtype='int32')

    x = layers.Embedding(
      output_dim=256,
      input_dim=voc_size,
      input_length=max_len)(pivot_input)
    x = layers.Activation('relu')(x)
    embedded_pivot = layers.Flatten()(x)

    encoder_model = Model(pivot_input, embedded_pivot)
    embedded_statement = encoder_model(statement_input)

    concat = layers.merge([embedded_pivot, embedded_statement], mode='concat')
    x = layers.Dropout(dropout)(concat)
    prediction = layers.Dense(1, activation='sigmoid')(x)

    model = Model([pivot_input, statement_input], prediction)
    return model

# Contains both the model definition function and the type of encoding needed.
MODELS = {
'jup_model': (jup_model, 'integer'),
}

### 2-layer 1D convolutional neural network 
***(CNN) with global maxpooling for sequence reduction. This model aims to determine the importance of local patterns of tokens.**

In [ ]:
%%writefile deepmath/deephol/holstep_baselines/jup_models/custom/unconditioned_model.py

"""Models for classifying proof steps without conditioning on the conjecture."""

###### your code below #####

def unconditioned_model(voc_size, max_len, dropout=0.5):
    """One branch embedding a statement. Binary classifier on top.
      Args:
        voc_size: size of the vocabulary for the input statements.
        max_len: maximum length for the input statements.
        dropout: Fraction of units to drop.
      Returns:
        A Keras model instance.
      """
    statement_input = layers.Input(shape=(max_len,), dtype='int32')

    x = layers.Embedding(
      output_dim=256,
      input_dim=voc_size,
      input_length=max_len)(statement_input)
    x = layers.Convolution1D(256, 7, activation='relu')(x)
    x = layers.MaxPooling1D(3)(x)
    x = layers.Convolution1D(256, 7, activation='relu')(x)
    embedded_statement = layers.GlobalMaxPooling1D()(x)

    x = layers.Dense(256, activation='relu')(embedded_statement)
    x = layers.Dropout(dropout)(x)
    prediction = layers.Dense(1, activation='sigmoid')(x)
    model = Model(statement_input, prediction)
    return model

# Contains both the model definition function and the type of encoding needed.
MODELS = {
'jup_model': (jup_model, 'integer'),
}

## Train Models

In [ ]:
%%bash

python holstep_baselines/main.py \
--model_name=jup_model \
--task_name=conditioned_classification \
--logdir=experiments/jup_model_experiment \
--source_dir=~/Downloads/holstep

# Selection Strategies
The action generator is responsible for building the search tree by choosing tactics to apply, and selecting premises. This task is where the advantages of machine learning can be realized. The action generator allows us to intelligently choose tactics and theorems to apply to our goals, greatly narrowing our search space. 

Tactic scoring and premise selection are handled by the abstract `Predictions` class located in [predictions.py](https://github.com/tensorflow/deepmath/blob/master/deepmath/deephol/predictions.py). New predictors can be implemented by subclassing the `Predictions` class, and implementing the following abstract methods.


## Goal Embedding 

**`_batch_goal_embedding(self, goals: List[Text]) -> BATCH_GOAL_EMB_TYPE`**  
Compute embeddings from a list of goals   
  
  

#### Parameters:
-   `goals: List[Text]`
List of goals as strings

#### Return Value:
-   `goal_embeddings: numpy.ndarray`  



In [ ]:
%%writefile deepmath/deephol/jup_predict/custom/goal_emb.py

###### your code below #####

import numpy as np

def _batch_goal_embedding(predictor, goals) -> BATCH_GOAL_EMB_TYPE:
    return np.array([[goal.__hash__(), 0] for goal in goals])

## Theorem Embedding 

**` _batch_thm_embedding(self, thms: List[Text]) -> BATCH_THM_EMB_TYPE`**  
From a list of string theorems, computes and returns their embeddings.  

#### Parameters
-   `thms: List[Text]`  
List of theorems as strings

#### Return Value
- `theorem_embeddings: numpy.ndarray`

In [ ]:
%%writefile deepmath/deephol/jup_predict/custom/thm_emb.py

###### your code below #####

import numpy as np

def _batch_thm_embedding(predictor, thms):
    """From a list of string theorems, compute and return their embeddings."""
    return np.array([[thm.__hash__(), 1] for thm in thms])

## Proof State 

### `proof_state_from_search(self, node: proof_search_tree.ProofSearchNode) -> ProofState:`
Convert from `proof_search_tree.ProofSearchNode` to proof state.

#### Parameters
-   `node: proof_search_tree.ProofSearchNode`

#### Return Value
-   `proof_state: predictions.ProofState`

In [ ]:
%%writefile deepmath/deephol/jup_predict/custom/st_search.py

###### your code below #####

from deepmath.deephol import predictions

def _proof_state_from_search(predictor, node):
    return predictions.ProofState(goal='goal')

## Proof State Embedding 

### `proof_state_embedding(self, state: ProofState) -> EmbProofState:`
From a proof state, computes and returns embeddings of each component.  

#### Parameters
-   `state: predictions.ProofState`

#### Return Value
-   `proof_state_embedding: predictions.EmbProofState`


In [ ]:
%%writefile deepmath/deephol/jup_predict/custom/st_emb.py

###### your code below #####

from deepmath.deephol import predictions

def _proof_state_embedding(predictor, state):
    return predictions.EmbProofState(*[[x.__hash__(), 2] for x in state])

## Proof State Encoding 

### `proof_state_encoding(self, state_emb: EmbProofState) -> STATE_ENC_TYPE:`
From an embedding of a proof state, computes and returns its encoding.

#### Parameters
-   `state_emb: predictions.EmbProofState`

#### Return Value
-   `state_encoding: numpy.ndarray`


In [ ]:
%%writefile deepmath/deephol/jup_predict/custom/st_enc.py

###### your code below #####

def _proof_state_encoding(predictor, state_emb):
    return state_emb.goal_emb

## Tactic Selection

### `_batch_tactic_scores(self, state_encodings: List[STATE_ENC_TYPE]) -> np.ndarray:`
Predicts tactic probabilities for a batch of goals.  

#### Parameters
-   `state_encodings: List[numpy.ndarray[TODO]]`

#### Return Value
-   `tactic_scores: numpy.ndarray[batch_size, num_tactics]`  

2D array of dimension \[batch_size, num_tactics\] representing tactic probabilities. Batch size is the length of `goal_embeddings`

In [ ]:
%%writefile deepmath/deephol/jup_predict/custom/tac_sc.py

###### your code below #####

import numpy as np

def _batch_tactic_scores(predictor, goal_embeddings):
    num_tactics = 41
    return np.array([[np.sum(emb)] + [.0]*40 for emb in goal_embeddings]) 

## Premise Selection

### ` _batch_thm_scores(self, state_encodings: List[STATE_ENC_TYPE], thm_embeddings: BATCH_THM_EMB_TYPE, tactic_id: Optional[int] = None) -> List[float]:` 
Predict relevance scores for goal, theorem pairs.  

#### Parameters
-   `state_encodings: List[numpy.ndarray]`
-   `theorem_embeddings: numpy.ndarray`
-   `tactic_id: int` (optional)

#### Return Value
-   `tactic_scores: List[float]`

In [ ]:
%%writefile deepmath/deephol/jup_predict/custom/thm_sc.py

###### your code below #####

import numpy as np

def _batch_thm_scores(predictor, goal_embeddings, thm_embeddings, tactic_id=None):
    if tactic_id is not None:
        c = 3.0 * float(tactic_id)
    else:
        c = 2.0
    return np.array([
          np.sum(e1) + c * np.sum(e2)
          for (e1, e2) in zip(goal_embeddings, thm_embeddings)
  ])

---


---
---
---

# Run DeepHOL

In [20]:
!python bazel-bin/deepmath/deephol/main --prover_options=data/configuration/prover_options.textpb --output=proof_logs.textpbs --proof_assistant_server_address=172.18.0.2:2000



	->  Entered main with args ['--prover_options=data/configuration/prover_options.textpb', '--output=proof_logs.textpbs', '--proof_assistant_server_address=172.18.0.2:2000'] 


/home/aaron/Source/HOList_conda/deepmath/bazel-bin/deepmath/deephol/main.runfiles/org_tensorflow/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aaron/Source/HOList_conda/deepmath/bazel-bin/deepmath/deephol/main.runfiles/org_tensorflow/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aaron/Source/HOList_conda/deepmath/bazel-bin/deepmath/deephol/main.runfiles/org_tensorflow/tensorflow/python/fram

I1202 13:22:55.078919 139969008183104 action_generator.py:341] thm_ranked: [(6.866725546611365e+18, 'THM 3333944427626911849'), (6.866725546611364e+18, 'THM 999940722663107508'), (6.866725546611364e+18, 'THM 999537060255340139'), (6.866725546611364e+18, 'THM 999460751669207476'), (6.866725546611364e+18, 'THM 998872802184983462'), (6.866725546611364e+18, 'THM 998577541292119328'), (6.866725546611364e+18, 'THM 998406517579130708'), (6.866725546611364e+18, 'THM 997761529271075455'), (6.866725546611364e+18, 'THM 997023046976021968'), (6.866725546611364e+18, 'THM 996274648282274911'), (6.866725546611364e+18, 'THM 996248022536094951'), (6.866725546611364e+18, 'THM 995599454161281824'), (6.866725546611364e+18, 'THM 995257399645182660'), (6.866725546611364e+18, 'THM 993831307850392473'), (6.866725546611364e+18, 'THM 993245494161554950'), (6.866725546611364e+18, 'THM 993193919539023191'), (6.866725546611364e+18, 'THM 991731817807010871'), (6.866725546611364e+18, 'THM 991719174871322337'), (6.86

I1202 13:22:55.280072 139969008183104 action_generator.py:341] thm_ranked: [(6.866725546611365e+18, 'THM 3333944427626911849'), (6.866725546611364e+18, 'THM 999940722663107508'), (6.866725546611364e+18, 'THM 999537060255340139'), (6.866725546611364e+18, 'THM 999460751669207476'), (6.866725546611364e+18, 'THM 998872802184983462'), (6.866725546611364e+18, 'THM 998577541292119328'), (6.866725546611364e+18, 'THM 998406517579130708'), (6.866725546611364e+18, 'THM 997761529271075455'), (6.866725546611364e+18, 'THM 997023046976021968'), (6.866725546611364e+18, 'THM 996274648282274911'), (6.866725546611364e+18, 'THM 996248022536094951'), (6.866725546611364e+18, 'THM 995599454161281824'), (6.866725546611364e+18, 'THM 995257399645182660'), (6.866725546611364e+18, 'THM 993831307850392473'), (6.866725546611364e+18, 'THM 993245494161554950'), (6.866725546611364e+18, 'THM 993193919539023191'), (6.866725546611364e+18, 'THM 991731817807010871'), (6.866725546611364e+18, 'THM 991719174871322337'), (6.86

I1202 13:22:55.490421 139969008183104 action_generator.py:341] thm_ranked: [(6.866725546611365e+18, 'THM 3333944427626911849'), (6.866725546611364e+18, 'THM 999940722663107508'), (6.866725546611364e+18, 'THM 999537060255340139'), (6.866725546611364e+18, 'THM 999460751669207476'), (6.866725546611364e+18, 'THM 998872802184983462'), (6.866725546611364e+18, 'THM 998577541292119328'), (6.866725546611364e+18, 'THM 998406517579130708'), (6.866725546611364e+18, 'THM 997761529271075455'), (6.866725546611364e+18, 'THM 997023046976021968'), (6.866725546611364e+18, 'THM 996274648282274911'), (6.866725546611364e+18, 'THM 996248022536094951'), (6.866725546611364e+18, 'THM 995599454161281824'), (6.866725546611364e+18, 'THM 995257399645182660'), (6.866725546611364e+18, 'THM 993831307850392473'), (6.866725546611364e+18, 'THM 993245494161554950'), (6.866725546611364e+18, 'THM 993193919539023191'), (6.866725546611364e+18, 'THM 991731817807010871'), (6.866725546611364e+18, 'THM 991719174871322337'), (6.86

I1202 13:22:55.799842 139969008183104 action_generator.py:341] thm_ranked: [(6.866725546611365e+18, 'THM 3333944427626911849'), (6.866725546611364e+18, 'THM 999940722663107508'), (6.866725546611364e+18, 'THM 999537060255340139'), (6.866725546611364e+18, 'THM 999460751669207476'), (6.866725546611364e+18, 'THM 998872802184983462'), (6.866725546611364e+18, 'THM 998577541292119328'), (6.866725546611364e+18, 'THM 998406517579130708'), (6.866725546611364e+18, 'THM 997761529271075455'), (6.866725546611364e+18, 'THM 997023046976021968'), (6.866725546611364e+18, 'THM 996274648282274911'), (6.866725546611364e+18, 'THM 996248022536094951'), (6.866725546611364e+18, 'THM 995599454161281824'), (6.866725546611364e+18, 'THM 995257399645182660'), (6.866725546611364e+18, 'THM 993831307850392473'), (6.866725546611364e+18, 'THM 993245494161554950'), (6.866725546611364e+18, 'THM 993193919539023191'), (6.866725546611364e+18, 'THM 991731817807010871'), (6.866725546611364e+18, 'THM 991719174871322337'), (6.86

...

I1202 13:31:53.606118 139969008183104 action_generator.py:341] thm_ranked: [(2.5319729042423455e+18, 'THM 811502063986834082'), (2.5319729042423455e+18, 'THM 3552238888230228107'), (2.5319729042423455e+18, 'THM 3333944427626911849'), (2.5319729042423455e+18, 'THM 1950647546301254424'), (2.531972904242345e+18, 'THM 999940722663107508'), (2.531972904242345e+18, 'THM 999537060255340139'), (2.531972904242345e+18, 'THM 999460751669207476'), (2.531972904242345e+18, 'THM 998872802184983462'), (2.531972904242345e+18, 'THM 998577541292119328'), (2.531972904242345e+18, 'THM 998406517579130708'), (2.531972904242345e+18, 'THM 997761529271075455'), (2.531972904242345e+18, 'THM 997023046976021968'), (2.531972904242345e+18, 'THM 996274648282274911'), (2.531972904242345e+18, 'THM 996248022536094951'), (2.531972904242345e+18, 'THM 995599454161281824'), (2.531972904242345e+18, 'THM 995257399645182660'), (2.531972904242345e+18, 'THM 993831307850392473'), (2.531972904242345e+18, 'THM 993245494161554950'),

I1202 13:31:54.238944 139969008183104 action_generator.py:341] thm_ranked: [(2.5319729042423455e+18, 'THM 811502063986834082'), (2.5319729042423455e+18, 'THM 3552238888230228107'), (2.5319729042423455e+18, 'THM 3333944427626911849'), (2.5319729042423455e+18, 'THM 1950647546301254424'), (2.531972904242345e+18, 'THM 999940722663107508'), (2.531972904242345e+18, 'THM 999537060255340139'), (2.531972904242345e+18, 'THM 999460751669207476'), (2.531972904242345e+18, 'THM 998872802184983462'), (2.531972904242345e+18, 'THM 998577541292119328'), (2.531972904242345e+18, 'THM 998406517579130708'), (2.531972904242345e+18, 'THM 997761529271075455'), (2.531972904242345e+18, 'THM 997023046976021968'), (2.531972904242345e+18, 'THM 996274648282274911'), (2.531972904242345e+18, 'THM 996248022536094951'), (2.531972904242345e+18, 'THM 995599454161281824'), (2.531972904242345e+18, 'THM 995257399645182660'), (2.531972904242345e+18, 'THM 993831307850392473'), (2.531972904242345e+18, 'THM 993245494161554950'),

I1202 13:31:55.052038 139969008183104 action_generator.py:341] thm_ranked: [(2.5319729042423455e+18, 'THM 811502063986834082'), (2.5319729042423455e+18, 'THM 3552238888230228107'), (2.5319729042423455e+18, 'THM 3333944427626911849'), (2.5319729042423455e+18, 'THM 1950647546301254424'), (2.531972904242345e+18, 'THM 999940722663107508'), (2.531972904242345e+18, 'THM 999537060255340139'), (2.531972904242345e+18, 'THM 999460751669207476'), (2.531972904242345e+18, 'THM 998872802184983462'), (2.531972904242345e+18, 'THM 998577541292119328'), (2.531972904242345e+18, 'THM 998406517579130708'), (2.531972904242345e+18, 'THM 997761529271075455'), (2.531972904242345e+18, 'THM 997023046976021968'), (2.531972904242345e+18, 'THM 996274648282274911'), (2.531972904242345e+18, 'THM 996248022536094951'), (2.531972904242345e+18, 'THM 995599454161281824'), (2.531972904242345e+18, 'THM 995257399645182660'), (2.531972904242345e+18, 'THM 993831307850392473'), (2.531972904242345e+18, 'THM 993245494161554950'),

I1202 13:31:55.684743 139969008183104 action_generator.py:341] thm_ranked: [(2.5319729042423455e+18, 'THM 811502063986834082'), (2.5319729042423455e+18, 'THM 3552238888230228107'), (2.5319729042423455e+18, 'THM 3333944427626911849'), (2.5319729042423455e+18, 'THM 1950647546301254424'), (2.531972904242345e+18, 'THM 999940722663107508'), (2.531972904242345e+18, 'THM 999537060255340139'), (2.531972904242345e+18, 'THM 999460751669207476'), (2.531972904242345e+18, 'THM 998872802184983462'), (2.531972904242345e+18, 'THM 998577541292119328'), (2.531972904242345e+18, 'THM 998406517579130708'), (2.531972904242345e+18, 'THM 997761529271075455'), (2.531972904242345e+18, 'THM 997023046976021968'), (2.531972904242345e+18, 'THM 996274648282274911'), (2.531972904242345e+18, 'THM 996248022536094951'), (2.531972904242345e+18, 'THM 995599454161281824'), (2.531972904242345e+18, 'THM 995257399645182660'), (2.531972904242345e+18, 'THM 993831307850392473'), (2.531972904242345e+18, 'THM 993245494161554950'),

I1202 13:31:56.141624 139969008183104 proof_search_tree.py:263] Tactic application failed: ARITH_TAC, Failure("Re-fail Calc_rat.REAL_ARITH: linear_ineqs: no contradiction")
I1202 13:31:56.208302 139969008183104 proof_search_tree.py:263] Tactic application failed: DISJ_CASES_TAC THM 811502063986834082, Failure("DISJ_CASES_TAC")
I1202 13:31:56.265360 139969008183104 proof_search_tree.py:263] Tactic application failed: REAL_ARITH_TAC, Failure("Re-fail Calc_rat.REAL_ARITH: linear_ineqs: no contradiction")
I1202 13:31:56.340240 139969008183104 proof_search_tree.py:278] Tactic REWRITE_TAC [ THM 811502063986834082 ; THM 3552238888230228107 ; THM 3333944427626911849 ; THM 1950647546301254424 ; THM 999940722663107508 ; THM 999537060255340139 ; THM 999460751669207476 ; THM 998872802184983462 ; THM 998577541292119328 ; THM 998406517579130708 ; THM 997761529271075455 ; THM 997023046976021968 ; THM 996274648282274911 ; THM 996248022536094951 ; THM 995599454161281824 ; THM 995257399645182660 ; THM 9

# Related Work

## Papers
* [HOList: An Environment for Machine Learning of Higher-Order Theorem Proving](https://arxiv.org/pdf/1904.03241.pdf)  
* [HolStep: A Machine Learning Dataset for Higher-order Logic Theorem Proving](https://arxiv.org/abs/1703.00426)
* [Deep Network Guided Proof Search](Deep Network Guided Proof Search)
* [GamePad: A Learning Environment for Theorem Proving](https://arxiv.org/abs/1806.00608)
* [Machine Learning for First-Order Theorem Proving](https://link.springer.com/article/10.1007%2Fs10817-014-9301-5)
* [Reinforcement Learning of Theorem Proving](https://arxiv.org/abs/1805.07563)
* [Learning to Prove Theorems via Interacting with Proof Assistants](https://arxiv.org/pdf/1905.09381.pdf)  

## Resources 
* [HOList Site](https://sites.google.com/view/holist/home)
* [Deepmath Repository](https://github.com/tensorflow/deepmath) 
* [HOL-Light Repository](https://github.com/brain-research/hol-light)
* [CoqGym](https://github.com/princeton-vl/CoqGym)



# About This Project
This tutorial was made as part of an undergraduate research project at University of Central Florida, supervised by Dr. Sumit Jha.

# Contact
* Aaron Hadley: aahadley1@gmail.com
* Tyler McFadden: neruelin@gmail.com
* Jordan Starkey: jordanstarkey95@gmail.com
* Sumit Jha: Sumit.Jha@ucf.edu  

# Acknowledgements
We are especially thankful to the following individuals, who have been incredibly responsive and helpful in answering questions throughout this project.  

*Dr. Christian Szegedy, Dr. Sarah Loos, Kaiyu Yang, Dr. Jason Rute*